# Air Conditioner RAG System with Ollama
## ShopSmart E-commerce Intelligence System

This notebook demonstrates the RAG system using local Ollama models:
- Phi-3
- Llama3  
- Gemma2

**Dataset**: Air Conditioners from Amazon

## 1. Setup and Imports

In [ ]:
import sys
sys.path.append('..')

import yaml
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from src.data_loader import AmazonProductDataLoader
from src.vector_store import VectorStore
from src.ollama_handler import OllamaMultiLLMManager
from src.rag_system import RAGSystem
from src.evaluation import RAGEvaluator
from src.questions import get_all_questions

warnings.filterwarnings('ignore')

print("✓ All imports successful!")

## 2. Load Configuration

In [ ]:
with open('../config.yaml', 'r') as f:
    config = yaml.safe_load(f)

print("Configuration loaded successfully!")
print(f"Dataset path: {config['dataset']['path']}")
print(f"Embedding model: {config['embedding']['model_name']}")
print(f"Ollama URL: {config['ollama']['base_url']}")
print(f"\nConfigured LLMs: {list(config['llms'].keys())}")

## 3. Explore Air Conditioner Dataset

In [ ]:
# Load dataset
data_loader = AmazonProductDataLoader(config['dataset']['path'])
df = data_loader.load_data()

print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")

# Display first few rows
df.head()

In [ ]:
# Get statistics
stats = data_loader.get_statistics()

print("Dataset Statistics:")
print(f"Total products: {stats['total_products']}")
print(f"\nColumns: {stats['columns']}")

In [ ]:
# Visualize price distribution
import re

def parse_price(price_str):
    if pd.isna(price_str):
        return 0
    cleaned = re.sub(r'[^\d.]', '', str(price_str))
    return float(cleaned) if cleaned else 0

df['price_numeric'] = df['discount_price'].apply(parse_price)
df_clean = df[df['price_numeric'] > 0]

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(df_clean['price_numeric'], bins=30, edgecolor='black')
plt.xlabel('Price (₹)')
plt.ylabel('Count')
plt.title('Price Distribution of Air Conditioners')

plt.subplot(1, 2, 2)
plt.boxplot(df_clean['price_numeric'])
plt.ylabel('Price (₹)')
plt.title('Price Box Plot')

plt.tight_layout()
plt.show()

print(f"Price range: ₹{df_clean['price_numeric'].min():.2f} - ₹{df_clean['price_numeric'].max():.2f}")
print(f"Average price: ₹{df_clean['price_numeric'].mean():.2f}")
print(f"Median price: ₹{df_clean['price_numeric'].median():.2f}")

## 4. Build Vector Index

In [ ]:
# Preprocess data
df = data_loader.preprocess_data()
documents = data_loader.create_documents()

print(f"Created {len(documents)} documents")
print(f"\nSample document:")
print(documents[0]['text'][:400])

In [ ]:
# Create vector store
vector_store = VectorStore(
    embedding_model_name=config['embedding']['model_name'],
    index_path=config['vector_db']['index_path'],
    use_faiss=True
)

# Create embeddings
print("Creating embeddings... (this may take a minute)")
embeddings = vector_store.create_embeddings(documents, batch_size=32)
print(f"Embeddings shape: {embeddings.shape}")

In [ ]:
# Build and save index
vector_store.build_index()
print("Vector index built!")

vector_store.save_index()
print("Index saved!")

## 5. Test Retrieval

In [ ]:
# Test query
test_query = "Best budget 1.5 ton inverter AC with good energy rating"

results = vector_store.search(test_query, top_k=5)

print(f"Query: {test_query}\n")
print("Top 5 Results:\n")

for i, result in enumerate(results):
    print(f"{i+1}. Score: {result['score']:.3f}")
    print(f"   {result['document']['text'][:250]}...\n")

## 6. Load Ollama LLMs

**Important**: Make sure Ollama is running and you have downloaded the models:
```bash
ollama pull phi3
ollama pull llama3
ollama pull gemma2
```

In [ ]:
# Load Ollama LLM Manager
llm_manager = OllamaMultiLLMManager(
    llm_configs=config['llms'],
    base_url=config['ollama']['base_url']
)

print("Ollama LLM Manager initialized!")

In [ ]:
# Load one model for testing (Phi-3 is fastest/smallest)
try:
    llm_manager.load_llm('phi3')
    print("✓ Phi-3 loaded successfully!")
except Exception as e:
    print(f"Error loading Phi-3: {e}")
    print("\nMake sure:")
    print("1. Ollama is running: ollama serve")
    print("2. Model is pulled: ollama pull phi3")

## 7. Create RAG System

In [ ]:
rag_system = RAGSystem(
    vector_store=vector_store,
    llm_manager=llm_manager,
    top_k=config['rag']['top_k'],
    system_prompt=config['prompts']['system_prompt'],
    qa_template=config['prompts']['qa_template']
)

print("RAG system ready!")

## 8. Test RAG System with Single Model

In [ ]:
# Test question
question = "What are the best value air conditioners under ₹35,000?"

result = rag_system.query(
    query=question,
    llm_name='phi3',
    return_context=True
)

print(f"Question: {question}\n")
print("="*80)
print("Answer (Phi-3):")
print("="*80)
print(result['answer'])

In [ ]:
# View retrieved context
print("Retrieved Documents:\n")
for i, doc in enumerate(result['retrieved_docs']):
    print(f"{i+1}. Score: {doc['score']:.3f}")
    print(f"   {doc['document']['text'][:200]}...\n")

## 9. Compare All Three Models

In [ ]:
# Load all models
print("Loading all models...\n")

for model_name in ['phi3', 'llama3', 'gemma2']:
    try:
        if model_name not in llm_manager.get_loaded_llms():
            llm_manager.load_llm(model_name)
            print(f"✓ {model_name} loaded")
    except Exception as e:
        print(f"✗ Failed to load {model_name}: {e}")

print(f"\nLoaded models: {llm_manager.get_loaded_llms()}")

In [ ]:
# Test same question with all models
test_question = "Which 1.5 ton inverter AC offers the best value under ₹40,000?"

print(f"Question: {test_question}\n")
print("="*80)

results_all = rag_system.query_all_llms(
    query=test_question,
    return_context=False
)

for model_name, result in results_all.items():
    print(f"\n{model_name.upper()} Response:")
    print("-"*80)
    print(result['answer'])
    print("="*80)

## 10. Test with Evaluation Questions

In [ ]:
# Get evaluation questions
questions = get_all_questions()

print(f"Total questions: {len(questions)}\n")

# Display first 5 questions
for q in questions[:5]:
    print(f"Q{q['id']}. {q['question']}")
    print(f"   Category: {q['category']} | Difficulty: {q['difficulty']}\n")

In [ ]:
# Test first 3 questions with one model
results_list = []

for q in questions[:3]:
    print(f"\nProcessing Q{q['id']}: {q['question']}")
    
    result = rag_system.query(
        query=q['question'],
        llm_name='phi3'
    )
    
    results_list.append({
        'question_id': q['id'],
        'question': q['question'],
        'category': q['category'],
        'answer': result['answer'],
        'num_retrieved': result['num_retrieved']
    })
    
    print(f"Answer: {result['answer'][:200]}...")

print("\nDone!")

In [ ]:
# Display results in DataFrame
results_df = pd.DataFrame(results_list)
results_df

## 11. Model Comparison Analysis

In [ ]:
# Compare models on a specific question
comparison_question = questions[4]['question']  # Value reasoning question

print(f"Question: {comparison_question}\n")

comparison_results = {}
import time

for model_name in llm_manager.get_loaded_llms():
    start_time = time.time()
    
    result = rag_system.query(
        query=comparison_question,
        llm_name=model_name
    )
    
    elapsed_time = time.time() - start_time
    
    comparison_results[model_name] = {
        'answer': result['answer'],
        'time': elapsed_time,
        'length': len(result['answer'].split())
    }

# Display comparison
for model_name, data in comparison_results.items():
    print(f"\n{model_name.upper()}:")
    print(f"Time: {data['time']:.2f}s | Words: {data['length']}")
    print("-"*80)
    print(data['answer'])
    print("="*80)

In [ ]:
# Visualize comparison metrics
import numpy as np

models = list(comparison_results.keys())
times = [comparison_results[m]['time'] for m in models]
lengths = [comparison_results[m]['length'] for m in models]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Response time
ax1.bar(models, times, color=['#FF6B6B', '#4ECDC4', '#45B7D1'])
ax1.set_ylabel('Response Time (seconds)')
ax1.set_title('Model Response Time Comparison')
ax1.grid(axis='y', alpha=0.3)

# Answer length
ax2.bar(models, lengths, color=['#FF6B6B', '#4ECDC4', '#45B7D1'])
ax2.set_ylabel('Answer Length (words)')
ax2.set_title('Answer Length Comparison')
ax2.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## 12. Interactive Query Function

In [ ]:
def ask_question(question, model='phi3', show_context=True):
    """Interactive query function"""
    result = rag_system.query(
        query=question,
        llm_name=model,
        return_context=show_context
    )
    
    print("="*80)
    print(f"Q: {question}")
    print("="*80)
    print(f"\nA ({model}): {result['answer']}\n")
    print("-"*80)
    print(f"Retrieved {result['num_retrieved']} documents")
    print("-"*80)
    
    if show_context:
        print("\nTop Retrieved Documents:")
        for i, doc in enumerate(result['retrieved_docs'][:3]):
            print(f"\n{i+1}. (Score: {doc['score']:.3f})")
            print(f"   {doc['document']['text'][:200]}...")
    
    return result

# Try it
ask_question("What is the cheapest 5-star rated AC?")

## 13. Save Results for Analysis

In [ ]:
# Save comparison results to CSV
import json
from datetime import datetime

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = f"../outputs/model_comparison_{timestamp}.json"

# Create outputs directory if needed
import os
os.makedirs('../outputs', exist_ok=True)

# Save results
with open(output_file, 'w') as f:
    json.dump(comparison_results, f, indent=2)

print(f"Results saved to: {output_file}")

## Next Steps

1. **Run Full Evaluation**: Test all 15 questions with all 3 models
2. **Analyze Results**: Compare accuracy, factuality, and reasoning
3. **Generate Report**: Create visualizations and insights
4. **Optimize Prompts**: Improve system and QA prompts for better results

Use the `main.py` script for automated evaluation:
```bash
python main.py --evaluate-all
```